In [1]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets loralib sentencepiece
# !pip install -q gradio
# !pip install -q ttsmms

In [2]:
# # For TTS

# !curl https://dl.fbaipublicfiles.com/mms/tts/ory.tar.gz --output ory.tar.gz #update lang
# !mkdir -p data && tar -xzf ory.tar.gz -C data/ #update langcode

In [ ]:
# https://www.odiagenai.org/blog/odiagenai-released-the-first-llm-for-the-low-resource-odia-language

In [4]:
transformers.__version__

'4.43.0.dev0'

In [8]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel, PeftConfig


# Load the base LLaMA model
base_model_name = "baffo32/decapoda-research-llama-7B-hf"
lora_weights_path = "OdiaGenAI/odiagenAI-bengali-lora-model-v1"

# tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Hey! It was indeed fixed for other models, but Llama is a bit specific, we call self.update_post_processor() which makes sure that the eos token is added. This should work without a bos token, but you can't have no unk_token, this is pretty much a requirement for all our tokenizers. Not sure I can fix this as raising an error would not be BC :/ would recommend doing this:

tokenizer = transformers.LlamaTokenizer.from_pretrained('baffo32/decapoda-research-llama-7B-hf', unk_token="<unk>") 
model = transformers.LlamaForCausalLM.from_pretrained(
    base_model_name,
    # load_in_8bit=False,
    quantization_config=transformers.BitsAndBytesConfig(load_in_8bit=True),
    torch_dtype = torch.float16,
    device_map = 'auto'
)
lora_model = PeftModel.from_pretrained(
    model, 
    lora_weights_path,
    force_download = True
)


/home/reasat/miniconda3/envs/phison/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:519: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly by `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation, and ensure your `input_ids` input does not have negative values.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/71.7M [00:00<?, ?B/s]

In [9]:
# from peft import PeftModel, PeftConfig

# # Path to the LoRA weights
# lora_weights_path = "OdiaGenAI/odiagenAI-bengali-lora-model-v1"

# # Load the LoRA configuration
# config = PeftConfig.from_pretrained(lora_weights_path)

# # Apply the LoRA weights to the base model
# lora_model = PeftModel.from_pretrained(model, lora_weights_path, config=config)
# lora_model = lora_model.cuda()

In [10]:
# # import torch
# from peft import PeftModel
# # from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig


# import torch
# # from peft import PeftModel
# import transformers
# import gradio as gr

# assert (
#     "LlamaTokenizer" in transformers._import_structure["models.llama"]
# ), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, BitsAndBytesConfig

In [11]:
# model_id = "OdiaGenAI/odiagenAI_llama7b_base_v1"
# device = "cuda:0"


# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# tokenizer = LlamaTokenizer.from_pretrained(model_id)
# model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")   # 4 bit model

In [12]:
# model.eval()
# if torch.__version__ >= "2":
#     model = torch.compile(model)

In [13]:
def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    print(inputs)
    input_ids = inputs["input_ids"].to(device)
    print(input_ids)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
        # print(generation_output)
    s = generation_output.sequences[0]
    print(s)
    output = tokenizer.decode(s, skip_special_tokens=True)
    print(output)

    return text
# input_text = gr.components.Textbox(
#             lines=2, label="Instruction", placeholder="Tell me about alpacas."
#         )
# # tts_button = gr.Button("Change to voice", elem_id="send-btn", visible=True)

# output_audio = gr.outputs.Audio(label="Output", type="filepath")

# g = gr.Interface(
#     fn=evaluate,
#     inputs=[
#         input_text,
#         gr.components.Textbox(lines=2, label="Input", placeholder="none"),
#         gr.components.Slider(minimum=0, maximum=1, value=0.1, label="Temperature"),
#         gr.components.Slider(minimum=0, maximum=1, value=0.75, label="Top p"),
#         gr.components.Slider(minimum=0, maximum=100, step=1, value=40, label="Top k"),
#         gr.components.Slider(minimum=1, maximum=4, step=1, value=4, label="Beams"),
#         gr.components.Slider(
#             minimum=1, maximum=512, step=1, value=128, label="Max tokens"
#         ),
#     ],
#     outputs=[
#         gr.inputs.Textbox(
#             lines=5,
#             label="Output",
#         ),
#         output_audio
#     ],
#     title="🌊🐢 OdiaGenAI-4-bit",
#     description="OdiaGenAI-4-bit is a 7B-parameter LLaMA model finetuned to follow Odia instructions. It is trained on the [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca) Odia translated dataset and makes use of the Huggingface LLaMA implementation. For more information, please visit [the project's website](https://www.odiagenai.org/).",
# )



# g.queue(concurrency_count=2)
# g.launch(debug=True)

In [81]:
def generate_prompt(prompt_template, question, tokenizer):
    return prompt_template.format(question, context, "")

In [82]:
def generate_response(model, tokenizer, prompt):
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # print(responses)
    response_start = responses.find("### Response:") + len("### Response:")
    response = responses[response_start:].strip()
    return response

In [83]:
# prompt_template = """Below is an instruction in Bengali language that describes a task, paired with an input also in Bengali language that provides further context. Write a response in Bengali language that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}
# """
prompt_template = """নীচে একটি কাজের বর্ণনা সহ বাংলায় একটি নির্দেশনা দেওয়া হয়েছে, যা আরও প্রসঙ্গ প্রদান করে একটি ইনপুট সহ বাংলায় রয়েছে। বাংলায় একটি উত্তর লিখুন যা অনুরোধটি উপযুক্তভাবে সম্পন্ন করে।

### নির্দেশনা:
{}

### ইনপুট:
{}

### উত্তর:
{}
"""

In [84]:
temperature=0.1
top_p=0.75
top_k=40
num_beams=4
max_new_tokens=128
device = 'cuda'
prompt = generate_prompt(prompt_template, question, context)
# prompt = 'তুমি যদি চাও তবে'
print('==========PROMPT============')
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")
# print(inputs)
input_ids = inputs["input_ids"].to(device)
# print(input_ids)
generation_config = transformers.GenerationConfig(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    num_beams=num_beams,
    max_new_tokens = max_new_tokens,
    do_sample = True
    # **kwargs,
)
with torch.no_grad():
    generation_output = lora_model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
    )
    # print(generation_output)
s = generation_output.sequences[0]
# print(s)
output = tokenizer.decode(s, skip_special_tokens=True)
print('==========RESPONSE============')
print(output)

==========PROMPT============
নীচে একটি কাজের বর্ণনা সহ বাংলায় একটি নির্দেশনা দেওয়া হয়েছে, যা আরও প্রসঙ্গ প্রদান করে একটি ইনপুট সহ বাংলায় রয়েছে। বাংলায় একটি উত্তর লিখুন যা অনুরোধটি উপযুক্তভাবে সম্পন্ন করে।

### নির্দেশনা:
ভারতীয় বাঙালি কথাসাহিত্যিক মহাশ্বেতা দেবীর মৃত্যু কবে হয় ?

### ইনপুট:
২০১৬ সালের ২৩ জুলাই হৃদরোগে আক্রান্ত হয়ে মহাশ্বেতা দেবী কলকাতার বেল ভিউ ক্লিনিকে ভর্তি হন। সেই বছরই ২৮ জুলাই একাধিক অঙ্গ বিকল হয়ে তাঁর মৃত্যু ঘটে। তিনি মধুমেহ, সেপ্টিসেমিয়া ও মূত্র সংক্রমণ রোগেও ভুগছিলেন।

### উত্তর:


==========RESPONSE============
 নীচে একটি কাজের বর্ণনা সহ বাংলায় একটি নির্দেশনা দেওয়া হয়েছে, যা আরও প্রসঙ্গ প্রদান করে একটি ইনপুট সহ বাংলায় রয়েছে। বাংলায় একটি উত্তর লিখুন যা অনুরোধটি উপযুক্তভাবে সম্পন্ন করে।

### নির্দেশনা:
ভারতীয় বাঙালি কথাসাহিত্যিক মহাশ্বেতা দেবীর মৃত্যু কবে হয় ?

### ইনপুট:
২০১৬ সালের ২৩ জুলাই হৃদরোগে আক্রান্ত হয়ে মহাশ্বেতা দেবী কলকাতার বেল ভিউ ক্লিনিকে ভর্তি হন। সেই বছরই ২৮ জুলাই একাধিক অঙ্গ বিকল হয়ে তাঁর মৃত্যু ঘটে। তিনি মধুমেহ, সেপ্টিসেমিয়া ও

In [85]:
generation_output.scores[10]

tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:3')